In [36]:
import numpy as np
import pandas as pd
import matplotlib as mp
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import GLS, GLSAR
from scipy.linalg import toeplitz
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn import linear_model as lm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

In [37]:
df = pd.read_csv(r"C:\Users\gabri\Downloads\dataset_lm.csv")

In [38]:
df.describe(include="all")

,Dependent Var,Explanatory Var #1,Explanatory Var #2,Explanatory Var #3,Explanatory Var #4,Explanatory Var #5,Explanatory Var #6,Explanatory Var #7,Explanatory Var #8,Explanatory Var #9,Explanatory Var #10,Explanatory Var #11,Explanatory Var #12,Explanatory Var #13,Explanatory Var #14,Explanatory Var #15
count,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000,422.000000
mean,67.838396,7.762058,59.725134,0.500000,-16.078411,50.133715,10.420808,61.887309,1.566351,-24.511691,49.728665,7.740874,60.765794,0.500000,-16.673762,50.066901
std,25.676960,7.013243,9.579112,0.500593,8.111197,6.975674,11.520725,14.819969,1.115260,8.500500,6.910690,7.102714,9.525835,0.500593,8.106466,6.794584
min,-2.997183,-4.832834,44.124858,0.000000,-29.774797,30.009511,-9.828552,34.093154,0.000000,-39.979696,26.436407,-4.949728,44.158200,0.000000,-29.750628,32.118882
25%,49.482037,1.720182,51.617692,0.000000,-23.792637,45.423422,-0.174835,49.952772,1.000000,-32.037707,45.283603,1.838210,52.849792,0.000000,-23.797763,45.825931
50%,66.780110,7.905455,59.735139,0.500000,-15.875481,50.093602,10.422513,62.554591,2.000000,-23.767548,49.842746,8.055297,60.773906,0.500000,-17.373619,50.038667
75%,86.801496,13.684104,67.870073,1.000000,-9.185191,54.896583,21.059713,74.441216,3.000000,-17.419390,54.576381,14.020396,69.262757,1.000000,-9.627544,54.962602
max,133.384795,19.973331,76.973576,1.000000,-2.060708,70.365951,29.994610,86.895006,3.000000,-10.129522,68.201681,19.992891,76.639179,1.000000,-2.003168,69.147818


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Dependent Var        422 non-null    float64
 1   Explanatory Var #1   422 non-null    float64
 2   Explanatory Var #2   422 non-null    float64
 3   Explanatory Var #3   422 non-null    int64  
 4   Explanatory Var #4   422 non-null    float64
 5   Explanatory Var #5   422 non-null    float64
 6   Explanatory Var #6   422 non-null    float64
 7   Explanatory Var #7   422 non-null    float64
 8   Explanatory Var #8   422 non-null    int64  
 9   Explanatory Var #9   422 non-null    float64
 10  Explanatory Var #10  422 non-null    float64
 11  Explanatory Var #11  422 non-null    float64
 12  Explanatory Var #12  422 non-null    float64
 13  Explanatory Var #13  422 non-null    int64  
 14  Explanatory Var #14  422 non-null    float64
 15  Explanatory Var #15  422 non-null    flo

In [40]:
df.head()

,Dependent Var,Explanatory Var #1,Explanatory Var #2,Explanatory Var #3,Explanatory Var #4,Explanatory Var #5,Explanatory Var #6,Explanatory Var #7,Explanatory Var #8,Explanatory Var #9,Explanatory Var #10,Explanatory Var #11,Explanatory Var #12,Explanatory Var #13,Explanatory Var #14,Explanatory Var #15
0,56.293458,13.698667,50.639873,0,-18.568035,45.121911,11.412501,56.410757,2,-12.281132,38.996909,-3.010548,49.195073,0,-21.153143,46.919314
1,58.473431,2.714725,65.845845,1,-25.105932,47.190213,10.080280,65.383107,3,-36.763585,51.654939,4.991111,45.591729,0,-6.474403,53.383508
2,94.195330,11.618072,65.072497,0,-7.897464,52.163036,11.057301,82.812717,0,-15.733547,48.913837,-2.457696,56.608806,0,-27.903299,48.515026
3,29.074583,0.818623,45.408996,1,-18.316132,54.356714,5.029029,48.812471,1,-12.825591,45.851732,14.974177,47.362594,1,-10.064411,55.266254
4,86.035569,9.077544,73.548021,0,-19.204165,47.186807,12.128134,62.520911,2,-13.804860,47.765904,9.593982,53.700562,0,-17.546302,48.150543


In [41]:
df.columns

Index(['Dependent Var', 'Explanatory Var #1', 'Explanatory Var #2',
       'Explanatory Var #3', 'Explanatory Var #4', 'Explanatory Var #5',
       'Explanatory Var #6', 'Explanatory Var #7', 'Explanatory Var #8',
       'Explanatory Var #9', 'Explanatory Var #10', 'Explanatory Var #11',
       'Explanatory Var #12', 'Explanatory Var #13', 'Explanatory Var #14',
       'Explanatory Var #15'],
      dtype='object')

In [42]:
y = df["Dependent Var"]
x = df[["Explanatory Var #1","Explanatory Var #2", "Explanatory Var #3",\
                                             "Explanatory Var #4","Explanatory Var #5", "Explanatory Var #6",\
                                             "Explanatory Var #7","Explanatory Var #8", "Explanatory Var #9",\
                                             "Explanatory Var #10","Explanatory Var #11", "Explanatory Var #12",\
                                             "Explanatory Var #13","Explanatory Var #14", "Explanatory Var #15"]]

In [43]:
x_updated = sm.add_constant(x)
model_1 = sm.OLS(y, x_updated).fit()
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Dependent Var   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.646e+30
Date:                Mon, 23 Oct 2023   Prob (F-statistic):               0.00
Time:                        21:30:56   Log-Likelihood:                 12017.
No. Observations:                 422   AIC:                        -2.400e+04
Df Residuals:                     406   BIC:                        -2.394e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  32.0000   9.29e-14   3.45e+14      0.000      32.000      32.000
Explanatory Var #1      1.3000    7.5e-16   1.73e+15      0.000       1.300       1.300
Explanatory Var #2      1.7000   5.45e-16   3.12e+15      0.000       1.700       1.700
Explanatory Var #3      6.2000   1.05e-14   5.93e+14      0.000       6.200       6.200
Explanatory Var #4      2.1000   6.44e-16   3.26e+15      0.000       2.100       2.100
Explanatory Var #5     -0.9000   7.55e-16  -1.19e+15      0.000      -0.900      -0.900
Explanatory Var #6  -9.563e-16   4.55e-16     -2.100      0.036   -1.85e-15    -6.1e-17
Explanatory Var #7  -3.986e-16   3.52e-16     -1.133      0.258   -1.09e-15    2.93e-16
Explanatory Var #8  -3.331e-15   4.78e-15     -0.697      0.486   -1.27e-14    6.07e-15
Explanatory Var #9   1.825e-15   6.11e-16      2.986      0.003    6.23e-16    3.03e-15
Explanatory Var #10 -2.039e-15   7.62e-16     -2.676      0.008   -3.54e-15   -5.41e-16
Explanatory Var #11 -2.186e-16   7.37e-16     -0.297      0.767   -1.67e-15    1.23e-15
Explanatory Var #12 -1.102e-15   5.51e-16     -1.998      0.046   -2.19e-15   -1.79e-17
Explanatory Var #13 -2.931e-14   1.06e-14     -2.775      0.006   -5.01e-14   -8.55e-15
Explanatory Var #14   1.76e-15   6.48e-16      2.714      0.007    4.85e-16    3.03e-15
Explanatory Var #15  1.141e-15   7.72e-16      1.478      0.140   -3.76e-16    2.66e-15
==============================================================================
Omnibus:                        0.215   Durbin-Watson:                   0.386
Prob(Omnibus):                  0.898   Jarque-Bera (JB):                0.336
Skew:                          -0.008   Prob(JB):                        0.845
Kurtosis:                       2.863   Cond. No.                     2.55e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.55e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [44]:
error_values = y - model_1.predict(x_updated)
error_values

0     -1.350031e-13
1     -1.065814e-13
2     -8.526513e-14
3     -1.243450e-13
4     -9.947598e-14
           ...     
417   -5.684342e-14
418   -2.842171e-14
419   -9.237056e-14
420    2.842171e-14
421   -8.526513e-14
Length: 422, dtype: float64

In [45]:
sigma = np.std(error_values)
sigma

4.640390102155463e-14

In [46]:
autocorrelation_values = acf(error_values, nlags=3)

autocorrelation_1 = autocorrelation_values[1]
autocorrelation_2 = autocorrelation_values[2]
autocorrelation_3 = autocorrelation_values[3]

print(autocorrelation_1)
print(autocorrelation_2)
print(autocorrelation_3)

0.030549160134438938
-0.023210085484719294
0.038378714513032916


In [47]:
cov_matrix = sigma**2*toeplitz(np.append([1,autocorrelation_1,autocorrelation_2,autocorrelation_3],np.zeros(418)))
sm.GLS(y,x_updated,cov_matrix).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:          Dependent Var   R-squared:                       1.000
Model:                            GLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.925e+29
Date:                Mon, 23 Oct 2023   Prob (F-statistic):               0.00
Time:                        21:30:57   Log-Likelihood:                 11862.
No. Observations:                 422   AIC:                        -2.369e+04
Df Residuals:                     406   BIC:                        -2.363e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  32.0000   1.34e-13   2.39e+14      0.000      32.000      32.000
Explanatory Var #1      1.3000   1.08e-15    1.2e+15      0.000       1.300       1.300
Explanatory Var #2      1.7000   7.86e-16   2.16e+15      0.000       1.700       1.700
Explanatory Var #3      6.2000   1.51e-14   4.12e+14      0.000       6.200       6.200
Explanatory Var #4      2.1000   9.27e-16   2.26e+15      0.000       2.100       2.100
Explanatory Var #5     -0.9000   1.09e-15  -8.26e+14      0.000      -0.900      -0.900
Explanatory Var #6  -1.099e-16   6.54e-16     -0.168      0.867    -1.4e-15    1.18e-15
Explanatory Var #7   -2.17e-15   5.07e-16     -4.283      0.000   -3.17e-15   -1.17e-15
Explanatory Var #8   1.527e-15   6.88e-15      0.222      0.824    -1.2e-14     1.5e-14
Explanatory Var #9   1.298e-15   8.81e-16      1.473      0.142   -4.35e-16    3.03e-15
Explanatory Var #10 -3.391e-16   1.09e-15     -0.310      0.757   -2.49e-15    1.81e-15
Explanatory Var #11 -3.227e-16   1.06e-15     -0.304      0.761   -2.41e-15    1.77e-15
Explanatory Var #12 -2.699e-15   7.93e-16     -3.404      0.001   -4.26e-15   -1.14e-15
Explanatory Var #13 -3.608e-16   1.52e-14     -0.024      0.981   -3.02e-14    2.95e-14
Explanatory Var #14 -3.609e-15   9.32e-16     -3.871      0.000   -5.44e-15   -1.78e-15
Explanatory Var #15   4.51e-16   1.11e-15      0.407      0.684   -1.73e-15    2.63e-15
==============================================================================
Omnibus:                        0.464   Durbin-Watson:                   0.334
Prob(Omnibus):                  0.793   Jarque-Bera (JB):                0.567
Skew:                           0.067   Prob(JB):                        0.753
Kurtosis:                       2.880   Cond. No.                     2.44e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [48]:
#Remove constant since it was not needed
X_new = x_updated.drop(columns=['const'])
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.5, random_state=42)

In [49]:
model_lasso = lm.Lasso(alpha=1).fit(X_train,y_train)
model_lasso.coef_

array([ 1.26972628,  1.68394638,  2.02626245,  2.08756512, -0.91746375,
       -0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.01314162,  0.        , -0.        ,  0.        , -0.03617731])

In [50]:
lasso_pred = model_lasso.predict(X_test)
mean_absolute_percent = (mean_absolute_percentage_error(y_test, lasso_pred)) * 100
print(f"Mean Absolute Percentage Error = {mean_absolute_percent}")

Mean Absolute Percentage Error = 4.432190198291579


In [51]:
alphas = [0.01, 0.05, 0.1, 0.3, 0.6, 0.8, 1.0, 5.0, 10.0]
mean_absolute_percent_values = []

for alpha in alphas:
    model_lasso = Lasso(alpha=alpha)
    model_lasso.fit(X_train, y_train)

    lasso_pred = model_lasso.predict(X_test)

    mean_absolute_percent = (mean_absolute_percentage_error(y_test, lasso_pred)) * 100
    mean_absolute_percent_values.append(mean_absolute_percent)

    print(f"Alpha: {alpha} - Mean Absolute Percentage Error: {mean_absolute_percent}")

Alpha: 0.01 - Mean Absolute Percentage Error: 0.0442988116334597
Alpha: 0.05 - Mean Absolute Percentage Error: 0.22161319995302747
Alpha: 0.1 - Mean Absolute Percentage Error: 0.443231865495802
Alpha: 0.3 - Mean Absolute Percentage Error: 1.3296605608499326
Alpha: 0.6 - Mean Absolute Percentage Error: 2.659320369845574
Alpha: 0.8 - Mean Absolute Percentage Error: 3.5457572613994905
Alpha: 1.0 - Mean Absolute Percentage Error: 4.432190198291579
Alpha: 5.0 - Mean Absolute Percentage Error: 7.165880967699969
Alpha: 10.0 - Mean Absolute Percentage Error: 8.587094351590425
